In [ ]:
import pandas as pd
import matplotlib

In [ ]:
%run data.ipynb
%run ../modules/preprocessing.ipynb
%run ../modules/cds.ipynb
%run ../modules/utils.ipynb

In [ ]:
df1 = load_uc_glu(1) # Glu May control data

In [ ]:
df5 = load_uc2(5, intact=True) # Glu June control data
# df17 = load_uc2(17)
df17 = pd.read_excel('/Users/bryan/Documents/ChenLab/10Jun2021/degraded/DrChen_tRNA_A_Glu_FA_17.xlsx')
df17 = thermo_df(df17)
df18 = load_uc2(18)

In [ ]:
df1_24k = df1[df1.Mass>23000]
df5_24k = df5[df5.Mass>24000]


In [ ]:
plotly_zone(df5_24k, y='Vol')
plotly_zone(df1_24k, y='Vol')

In [ ]:
ppm = 10
limit = 6
%run ../modules/preprocessing.ipynb

df_samples = [df5]
sample_names = ['5']
intacts = [24217.31, 24203.31, 24232.29, 24189.31, 24218.33]
# df_samples = [df2, df14, df8, df20]
# intacts = [24146, 24132, 24145, 24131, 24160, 24147, 24176]

intacts = np.array(intacts)
cols = [f'Group_{int(intact)}' for intact in intacts]
amino = 147.1
# intacts += amino - H2O # Amino Acid Glu
df_result = pd.DataFrame(columns=cols)
for idx, df_sample in enumerate(df_samples):
#     df_sample = df.copy()
#     df_sample.Mass -= 146.1 - H2O
    df_24k = df_sample[(df_sample.Mass>24000)&(df_sample.Mass<25000)&(df_sample.Vol>1E4)]
    df_24k = df_24k.sort_values('Vol', ascending=False)#.iloc[:50].copy()
    print('Top dots: ', df_24k.iloc[:10].Mass.tolist())
    df_count = df_24k.copy()
    df_count['cnt'] = 0
    df_founds = list()
    for intact in intacts:
#         intact += amino - 18.0106 # amino acid glu
        df_found = detect_combined_salts(df_24k, intact, limit=limit, ppm=ppm, amino=0)
        df_founds.append(df_found)
        
#         print(df_found.shape, df_count.shape)
        if not df_found.empty:
            df_found = df_found.sort_values('Vol', ascending=False)
            print('intact ', intact, 'hit ', df_found.Mass.unique().tolist())
#             print('intact ', intact - amino +H2O, 'hit ', df_found.Mass.unique().tolist()) #
#         df_count.loc[df_found.index, 'cnt'] += 1
#     print(df_count.sort_values('Vol', ascending=False))
#     print(df_founds)
    print(f'Sample {idx}')
    l = [df.Vol.sum()/1E3 for df in df_founds]
    l = np.round(l, 2)
    df_result.loc[idx] = l
    print(l)
    print(df_24k)
    df_24k.to_excel('~/Downloads/glu_adducts_#{}.xlsx'.format(sample_names[idx]))

print(df_result)

In [ ]:
# low, high = 3300, 6700
low, high = 10000, 12000
df_sample = df17.copy()
df_zone = df_sample[(df_sample.Mass>low)&(df_sample.Mass<high)]
df_zone = df_zone[(df_zone.RT>0.2)&(df_zone.RT<19)]

df_sample2 = df18.copy()
df_zone2 = df_sample2[(df_sample2.Mass>low)&(df_sample2.Mass<high)]
# df_zone2 = df_zone2[(df_zone2.RT>0.2)&(df_zone2.RT<15)]

# df_zone = df_sample[(df_sample.Mass>1000)&(df_sample.Mass<3000)]
# df_zone = df_zone[(df_zone.RT>0.2)&(df_zone.RT<3.0)]
plotly_zones(df_zone, df_zone2)
plotly_zones(df_zone, df_zone2, y='Vol')

In [ ]:
# Choose dots

idxs = list()
# create our callback function
def on_selection(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

def on_click(trace, points, selector):
#     print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

df_sample = df_zone2
f = zone_selection(df_sample, on_selection=on_selection, on_click=on_click)
f

In [ ]:
df_chosen = df_sample.iloc[idxs]
# df_zone = df_sample.drop(df_chosen.index)
df_chosen.shape

In [ ]:
df5p = df_chosen.copy()

In [ ]:
df3p = df_sample.drop(df5p.index)

In [ ]:
# plotly_zones(df5p, df3p)
plotly_zone(df5p)
plotly_zone(df5p, y='Vol')